In [1]:
%load_ext autoreload

In [2]:
import os, sys, ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

In [3]:
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULES_PATH = os.path.join(BASE_PATH, "modules")
METRICS_PATH = os.path.join(BASE_PATH, "metrics")
DATASET_PATH = os.path.join(BASE_PATH, "datasets")

In [4]:
sys.path.append(MODULES_PATH)

In [5]:
from active_learning import Pool, Dataset, ExperimentSuitMetrics, Config
from wrapper import McDropout, MomentPropagation
from models import setup_growth, fchollet_cnn
from data import BenchmarkData, DataSetType
from utils import MetricsTransformer

In [6]:
seed = 83152
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
val_set_size = 100
test_set_size = 10_000
initial_pool_size = 20

mnist = BenchmarkData(DataSetType.MNIST, os.path.join(DATASET_PATH, "mnist"), dtype=np.float32)
x_train, x_test, y_train, y_test = train_test_split(mnist.inputs, mnist.targets, test_size=test_set_size)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_set_size)

In [8]:
debug_metrics_path = os.path.join(METRICS_PATH, "debug")
experiment_suit_metrics = ExperimentSuitMetrics(debug_metrics_path)

In [9]:
meta_info = experiment_suit_metrics.read_meta()
initial_indices = meta_info["experiments"][0]["params"]["initial_indices"]

In [10]:
experiment_data = MetricsTransformer.load_from_dir(experiment_suit_metrics)
ast.literal_eval(experiment_data["indices_selected"][0])

[18568, 11906, 53942, 54900, 36581, 44449, 38883, 59540, 28363, 10799]

In [11]:
unlabeled_pool = Pool(x_train, y_train)
unlabeled_pool.annotate(initial_indices)

In [12]:
str(initial_indices)

'[836, 1034, 3278, 6563, 7656, 9477, 9510, 13228, 16375, 20170, 21573, 24311, 26170, 26971, 28686, 30281, 32193, 50056, 50696, 51906]'

In [13]:
unlabeled_pool.get_labeled_indices()

array([  836,  1034,  3278,  6563,  7656,  9477,  9510, 13228, 16375,
       20170, 21573, 24311, 26170, 26971, 28686, 30281, 32193, 50056,
       50696, 51906])

## Create Model and load weighting

In [14]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [15]:
num_classes = 10
base_model = fchollet_cnn(output=num_classes)

sample_size = 25
fit_params = {"epochs": 200, "batch_size": sample_size}
config = Config(
    fit=fit_params,
    query={"sample_size": sample_size},
    eval={"batch_size": 200, "sample_size": sample_size}
)

mc_model = McDropout(base_model, config=config)

optimizer = "adam"
loss = "sparse_categorical_crossentropy"
metrics = [keras.metrics.SparseCategoricalAccuracy()]
mc_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [16]:
mc_model.load_weights()

In [17]:
x_lab, y_lab = unlabeled_pool.get_labeled_data()

In [18]:
mc_model.fit(x_lab, y_lab)

Epoch 1/200
1/1 [==============================] - 0s 2ms/step - loss: 2.3212 - sparse_categorical_accuracy: 0.1000
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 2.2348 - sparse_categorical_accuracy: 0.2000
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 2.0682 - sparse_categorical_accuracy: 0.3500
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 1.8993 - sparse_categorical_accuracy: 0.6000
Epoch 5/200
1/1 [==============================] - 0s 1ms/step - loss: 1.6517 - sparse_categorical_accuracy: 0.6500
Epoch 6/200
1/1 [==============================] - 0s 1ms/step - loss: 1.4744 - sparse_categorical_accuracy: 0.8000
Epoch 7/200
1/1 [==============================] - 0s 1ms/step - loss: 1.1188 - sparse_categorical_accuracy: 0.8500
Epoch 8/200
1/1 [==============================] - 0s 920us/step - loss: 0.9952 - sparse_categorical_accuracy: 0.7000
Epoch 9/200
1/1 [==============================] - 0s 1ms/step - loss:

1/1 [==============================] - 0s 772us/step - loss: 9.1139e-04 - sparse_categorical_accuracy: 1.0000
Epoch 71/200
1/1 [==============================] - 0s 986us/step - loss: 0.0078 - sparse_categorical_accuracy: 1.0000
Epoch 72/200
1/1 [==============================] - 0s 1ms/step - loss: 7.7215e-04 - sparse_categorical_accuracy: 1.0000
Epoch 73/200
1/1 [==============================] - 0s 876us/step - loss: 0.0028 - sparse_categorical_accuracy: 1.0000
Epoch 74/200
1/1 [==============================] - 0s 860us/step - loss: 0.0037 - sparse_categorical_accuracy: 1.0000
Epoch 75/200
1/1 [==============================] - 0s 673us/step - loss: 9.6190e-04 - sparse_categorical_accuracy: 1.0000
Epoch 76/200
1/1 [==============================] - 0s 1ms/step - loss: 0.1231 - sparse_categorical_accuracy: 0.9500
Epoch 77/200
1/1 [==============================] - 0s 2ms/step - loss: 0.1818 - sparse_categorical_accuracy: 0.9500
Epoch 78/200
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 1ms/step - loss: 4.8082e-04 - sparse_categorical_accuracy: 1.0000
Epoch 139/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0152 - sparse_categorical_accuracy: 1.0000
Epoch 140/200
1/1 [==============================] - 0s 1ms/step - loss: 0.0024 - sparse_categorical_accuracy: 1.0000
Epoch 141/200
1/1 [==============================] - 0s 820us/step - loss: 6.9824e-04 - sparse_categorical_accuracy: 1.0000
Epoch 142/200
1/1 [==============================] - 0s 922us/step - loss: 0.0020 - sparse_categorical_accuracy: 1.0000
Epoch 143/200
1/1 [==============================] - 0s 602us/step - loss: 6.9893e-04 - sparse_categorical_accuracy: 1.0000
Epoch 144/200
1/1 [==============================] - 0s 1ms/step - loss: 9.4150e-05 - sparse_categorical_accuracy: 1.0000
Epoch 145/200
1/1 [==============================] - 0s 759us/step - loss: 0.0024 - sparse_categorical_accuracy: 1.0000
Epoch 146/200
1/1 [===========================

In [21]:
mc_model.evaluate(x_test, y_test, sample_size=25, batch_size=900)

{'loss': 1.8176742, 'accuracy': 0.526592}

In [22]:
mc_model.evaluate(x_test, y_test, sample_size=25, batch_size=900)

{'loss': 1.8234627, 'accuracy': 0.526928}

In [23]:
mc_model.evaluate(x_test, y_test, sample_size=25, batch_size=900)

{'loss': 1.8187547, 'accuracy': 0.527884}